The plan
- *basemodel (Done)*
- *Make RFC a function or class to be called (Done)*
- *RFC on scaled stances from last meeting (Done)*
- *RFC on scaled stances with extra features from last meeting (Done)*
- RFC on the previous with statics from the basemodel
- RFC more sensors
- Repeat last 5 with Hybrid model

Save the trained model in an individual folder.

### Importing libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import StandardScaler

import os

import sys

from tqdm.auto import tqdm

import shutil

from itertools import groupby

import matplotlib.pyplot as plt

from statistics import mean

import scipy.stats as stats
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.fftpack import fft
from scipy import signal

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Flatten, Conv1D, MaxPooling1D, Concatenate, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model


In [2]:
sys.path.insert(0,"/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester")

### Importing my Classes :)

In [3]:
parquet_path = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Data/parquet_files/"
models_path = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Models/"

In [4]:
# Data Class
import DataClass 
from DataClass import Data
# Random Forest Classifier Class
import RFClassifier
from RFClassifier import RFClassifier

In [5]:
# read prepared data
data = Data( 0, "", "",parquet_path)

## Basemodel

In [6]:
# the data used 
data.raw_statics_df.shape

(304, 129)

In [13]:
# splitting the data into train and test data
y = data.raw_statics_df['y']
X = data.raw_statics_df.drop(columns=['ID', 'level', 'y'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=64)

In [19]:
# Checking which RFC will work the best
rfc = RFClassifier(models_path,200,2)
ccs=rfc.multiple_training([5, 10, 20, 40, 100, 200],2,X_train, np.ravel(y_train), X_test, y_test)

Estimator = 5
Accuracy: 0.8553, Precision: 0.8462, Recall: 0.9821, F1: 0.9091
Estimator = 10
Accuracy: 0.8553, Precision: 0.8571, Recall: 0.9643, F1: 0.9076
Estimator = 20
Accuracy: 0.8289, Precision: 0.8525, Recall: 0.9286, F1: 0.8889
Estimator = 40
Accuracy: 0.8816, Precision: 0.8852, Recall: 0.9643, F1: 0.9231
Estimator = 100
Accuracy: 0.9211, Precision: 0.9167, Recall: 0.9821, F1: 0.9483
Estimator = 200
Accuracy: 0.9079, Precision: 0.9153, Recall: 0.9643, F1: 0.9391


In [14]:
# recreating the best model to be saved
rfc_100 = RFClassifier(models_path,100,2)
rfc_100.tain(X_train,np.ravel(y_train))
y_pred=rfc_100.predict(X_test)
scores=rfc_100.scores(y_test, y_pred)
# rfc_100.print_scores(scores[0],scores[1],scores[2],scores[3])
# saving the best model
rfc_100.save_model("RFC_raw_statics_100")

## RFC

### Scaled stances wih fwhm from the sum of forces from the right foot with 3 extra features
 'r_swings_time','r_max_peak','r_total_fwhm'

In [6]:
# input data and label data
df_1 = data.scaled_stances_statics_df[[ 'r_interp','r_swings_time','r_max_peak','r_total_fwhm']]
y_1 = data.scaled_stances_statics_df[[ 'y']]
# reformatting
df_1 = data.return_array_df(df_1,'r_interp',['r_swings_time','r_max_peak','r_total_fwhm'])
# splitting the data
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_1, y_1, random_state=42)

In [9]:
# Checking which RFC will work the best
rfc_1 = RFClassifier(models_path,200,2)
ccs=rfc_1.multiple_training([5, 10, 20, 40, 100, 200],2, X_train_1, y_train_1, X_test_1,  y_test_1 )

Estimator = 5
Accuracy: 0.9034, Precision: 0.9103, Recall: 0.9515, F1: 0.9304
Estimator = 10
Accuracy: 0.9223, Precision: 0.9313, Recall: 0.9560, F1: 0.9435
Estimator = 20
Accuracy: 0.9313, Precision: 0.9314, Recall: 0.9703, F1: 0.9504
Estimator = 40
Accuracy: 0.9390, Precision: 0.9362, Recall: 0.9767, F1: 0.9560
Estimator = 100
Accuracy: 0.9418, Precision: 0.9370, Recall: 0.9803, F1: 0.9581
Estimator = 200
Accuracy: 0.9414, Precision: 0.9356, Recall: 0.9812, F1: 0.9579


In [ ]:
# recreating the best model to be saved
rfc_1_100 = RFClassifier(models_path,100,2)
rfc_1_100.tain(X_train_1,np.ravel(y_train_1))
y_pred=rfc_1_100.predict(X_test_1)
scores=rfc_1_100.scores(y_test_1, y_pred)
rfc_1_100.print_scores(scores[0],scores[1],scores[2],scores[3])
# saving the best model
rfc_1_100.save_model("RFC_scaled_stances_100")

### Scaled stances wih fwhm from the sum of forces from the right foot with 6 extra features 
'r_swings_time','r_max_peak','r_total_fwhm', 'r_strides_time','r_peaks_toe','r_peaks_heel'

In [16]:
# input data and label data
df_2 = data.scaled_stances_statics_df[[ 'r_interp','r_swings_time','r_max_peak','r_total_fwhm','r_strides_time','r_peaks_toe','r_peaks_heel']]
y_2 = data.scaled_stances_statics_df[[ 'y']]
# reformatting
df_2 = data.return_array_df(df_2,'r_interp',['r_swings_time','r_max_peak','r_total_fwhm','r_strides_time','r_peaks_toe','r_peaks_heel'])
# splitting the data
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2, y_2, random_state=42)

In [17]:
# Checking which RFC will work the best
rfc_2 = RFClassifier(models_path,200,2)
ccs=rfc_2.multiple_training([5, 10, 20, 40, 100, 200],2, X_train_2, y_train_2, X_test_2,  y_test_2 )

Estimator = 5
Accuracy: 0.9276, Precision: 0.9308, Recall: 0.9651, F1: 0.9476
Estimator = 10
Accuracy: 0.9394, Precision: 0.9450, Recall: 0.9670, F1: 0.9559
Estimator = 20
Accuracy: 0.9497, Precision: 0.9475, Recall: 0.9803, F1: 0.9636
Estimator = 40
Accuracy: 0.9502, Precision: 0.9456, Recall: 0.9832, F1: 0.9640
Estimator = 100
Accuracy: 0.9537, Precision: 0.9461, Recall: 0.9880, F1: 0.9666
Estimator = 200
Accuracy: 0.9535, Precision: 0.9453, Recall: 0.9887, F1: 0.9665


In [ ]:
# recreating the best model to be saved
rfc_2_100 = RFClassifier(models_path,100,2)
rfc_2_100.tain(X_train_2,y_train_2)
y_pred_2=rfc_2_100.predict(X_test_2)
scores=rfc_2_100.scores(y_test_2, y_pred_2)
# rfc_2_100.print_scores(scores[0],scores[1],scores[2],scores[3])
# saving the best model
rfc_2_100.save_model("RFC_scaled_stances_6_features_100")